In [1]:
import tensorflow as tf
import numpy as np

**Matrix A (3x3):**
\begin{bmatrix}
x1 & x2 & x3\\
x4 & x5 & x6\\
x7 & x8 & x9
\end{bmatrix}

**Matrix B (3x4):**
\begin{bmatrix}
y1 & y2 & y3\\
y4 & y5 & y6\\
y7 & y8 & y9\\
y10 & y11 & y12
\end{bmatrix}

**First loop**
- Loop through every rows of A

**Second loop**
- Loop through every columns of B

**Third loop**
- Loop through every columns of A (or rows of B)

### Load dataset

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [3]:
m1 = X_train[:5].reshape(5, 784)
m1 = np.array(m1, dtype=np.float32)

m2 = np.random.rand(784, 10)
m2 = np.array(m2, dtype=np.float32)

In [4]:
def matmul(a, b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac == br
    
    result = np.zeros(shape=(ar, bc))
    
    for i in range(ar):
        for j in range(bc):
            for k in range(br):
                result[i, j] += a[i, k] * b[k, j]
                
    return result

In [5]:
%time t1=matmul(m1, m2)

CPU times: user 25.6 ms, sys: 866 µs, total: 26.5 ms
Wall time: 26.1 ms


In [6]:
#shape
t1.shape

(5, 10)

## Matrix multiplication 2
(Use element-wise to increase performance)

In [7]:
def matmul2(a, b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac == br
    
    result = np.zeros(shape=(ar, bc))
    
    for i in range(ar):
        for j in range(bc):
            result[i, j] = (a[i, :] * b[:, j]).sum()
                
    return result

In [8]:
# roughly 277 times faster
%timeit -n 10 _=matmul2(m1, m2)

275 µs ± 6.55 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Test broadcasting

In [9]:
m1.shape, m1[None, :].shape, m1[:, :, None, None].shape

((5, 784), (1, 5, 784), (5, 784, 1, 1))

In [10]:
m2.shape, m2[None, :].shape, m2[:, :, None, None].shape

((784, 10), (1, 784, 10), (784, 10, 1, 1))

In [11]:
# Can broadcast
m1 == np.zeros((5, 1))

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [12]:
m1 + np.zeros((1))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
# cannot broadcast.
# condition to broadcast
# (dimension21 == dimension11 && dimension22 == 1) ||
# (dimension21 == dimension22 == 1) ||
# (dimension22 == dimension12 && dimension21 == 1)
m1 == np.zeros((5, 2))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


False

## Matrix multiplication 3
(Use broadcasting to increase performance)

In [14]:
def matmul3(a, b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac == br
    
    result = np.zeros(shape=(ar, bc))
    
    for i in range(ar):
#         print((a[i][:, None] * b).sum(axis=0).shape)
        result[i] = (a[i][:, None] * b).sum(axis=0)
                
    return result

In [15]:
%timeit -n 10 _=matmul3(m1, m2)

125 µs ± 7.08 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
matmul3(m1, m2).shape

(5, 10)

## Einstein Summation

In [17]:
type(m1[0][0]), type(m2[0][0])

(numpy.float32, numpy.float32)

In [18]:
def matmul4(a, b): return tf.einsum('ik,kj->ij', a, b)

In [19]:
%timeit -n 10 _=matmul4(m1, m2)

The slowest run took 1150.04 times longer than the fastest. This could mean that an intermediate result is being cached.
83.7 ms ± 204 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Numpy matmul

In [20]:
def matmul5(a, b): return np.matmul(a, b);

In [21]:
%timeit -n 10 _=matmul5(m1, m2)

10 µs ± 2.66 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Tensorflow matmul

In [22]:
def matmul6(a, b): return tf.linalg.matmul(a, b);

In [23]:
%timeit -n 10 _=matmul6(m1, m2)

519 µs ± 25.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
